In [ ]:
import os
from openai import OpenAI

#ücretli
client = OpenAI(
    api_key="",

)

#ücretsiz - testler için github api key oluştur
# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
# Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# from openai import OpenAI

# client = OpenAI(
#     base_url="https://models.inference.ai.azure.com",
#     api_key="",
# )


In [ ]:
from pydantic import BaseModel


class EvaluationResponse(BaseModel):
    evaluation_a: list[str]
    grade_a: int
    grades_each_criteria_a: list[int]

    evaluation_b: list[str]
    grade_b: int
    grades_each_criteria_b: list[int]

    totalGrade: int



# print(response)

In [ ]:
def get_completion_resformat(prompt_user, model="gpt-4o", temperature=0.3, response_format = None):
  response = client.beta.chat.completions.parse(
      messages=[
          {
              "role": "system",
              "content": "You are a teaching assistant evaluating a student's answers about Operating Systems course for correctness and completeness."

          },
          {
              "role": "user",
              "content": prompt_user,
          }
      ],
      model=model,
      temperature=temperature,
      response_format = response_format
  )

  return response.choices[0].message.parsed

In [ ]:
import re
import openai
import pandas as pd
import os
from time import sleep

# Load students' answers from the file
file_path = '/content/Quiz 2 Papers & Answers(1-147).xlsx'
df = pd.read_excel(file_path, sheet_name = 'Sheet2')

# Define the path to the prompts folder
# script_dir = os.path.dirname(os.path.abspath(__file__))
prompts_dir = '/content/prompts/Quiz2'

# Load the question and grading details from the prompts folder
with open(os.path.join(prompts_dir, 'the_question.txt'), 'r') as f:
    the_question = f.read()

with open(os.path.join(prompts_dir, 'grading_prompt.txt'), 'r') as f:
    grading_prompt = f.read()

# Function to evaluate all parts of a student's answers in a single API call
def evaluate_answers_combined(answers, temp_p = 0.3, model_p = "gpt-4o"):
    # Combine answers for all parts into a single string

    temperature = temp_p
    model = model_p
    answers_combined = f"""
    Question 1a:
    {answers['1a']}

    Question 1b:
    {answers['1b']}
    """

    # Create the API prompt
    prompt = grading_prompt.format(the_question) + "\n\nStudent Answers:\n" + answers_combined
    # print(prompt)


    response = get_completion_resformat(prompt, model= model, temperature=temperature, response_format=EvaluationResponse)
    # sleep(0.01)

    # print(response)
    # Extract the response text
    evaluation_result = response



    return evaluation_result


In [ ]:

# @title Varsayılan başlık metni

from google.colab import files
# Define the temperatures and models you want to test

temperatures = [0.0, 0.5]
models = ['gpt-4o-mini', 'gpt-4o']  # use actual API model names

# Process each student's answers
results = []

for index, row in df.iterrows():
    print("row['STD_CODE']: {}".format(row['STD_CODE']))
    student = {
        'Name': row['Name'],
        'Email': row['Email'],
        'STD_CODE': row['STD_CODE']
    }

    answers = {
        '1a': row['Write the answer to PART A directly here. (You can copy the word)'],
        '1b': row['Write the answer to PART B  directly here. (You can copy the word)']
    }

    result_entry = {
        'Name': student['Name'],
        'Email': student['Email'],
        'STD_CODE': student['STD_CODE']
    }

    for model in models:
        for temp_p in temperatures:
            try:
                evaluation_result = evaluate_answers_combined(answers, temp_p, model)

                # Safe model name for column usage
                model_safe = model.replace(".", "").replace("-", "")
                suffix = f"_{temp_p}_{model_safe}"

                result_entry[f'Feedback_a{suffix}'] = evaluation_result.evaluation_a
                result_entry[f'Grade_a{suffix}'] = evaluation_result.grade_a
                result_entry[f'Feedback_b{suffix}'] = evaluation_result.evaluation_b
                result_entry[f'Grade_b{suffix}'] = evaluation_result.grade_b
                result_entry[f'Taken_grades_each_criteria_a{suffix}'] = evaluation_result.grades_each_criteria_a
                result_entry[f'Taken_grades_each_criteria_b{suffix}'] = evaluation_result.grades_each_criteria_b
                result_entry[f'Total_grade{suffix}'] = evaluation_result.totalGrade

            except Exception as e:
                print(f"Error evaluating {student['Name']} with {model} at temp {temp_p}: {e}")
                continue

    results.append(result_entry)

# Convert results to DataFrame and export
results_df = pd.DataFrame(results)
output_file = 'graded_students_answers_all_models_temps_Quiz2All.xlsx'
results_df.to_excel(output_file, index=False)
print(f"Grading completed and saved to {output_file}.")

# Trigger file download
files.download(output_file)

row['STD_CODE']: 1000
row['STD_CODE']: 1002
row['STD_CODE']: 1004
row['STD_CODE']: 1006
row['STD_CODE']: 1008
row['STD_CODE']: 1010
row['STD_CODE']: 1012
row['STD_CODE']: 1014
row['STD_CODE']: 1016
row['STD_CODE']: 1018
row['STD_CODE']: 1020
row['STD_CODE']: 1022
row['STD_CODE']: 1024
row['STD_CODE']: 1026
row['STD_CODE']: 1028
row['STD_CODE']: 1030
row['STD_CODE']: 1032
row['STD_CODE']: 1034
row['STD_CODE']: 1036
row['STD_CODE']: 1038
row['STD_CODE']: 1040
row['STD_CODE']: 1042
row['STD_CODE']: 1044
row['STD_CODE']: 1046
row['STD_CODE']: 1048
row['STD_CODE']: 1050
row['STD_CODE']: 1052
row['STD_CODE']: 1054
row['STD_CODE']: 1056
row['STD_CODE']: 1058
row['STD_CODE']: 1060
row['STD_CODE']: 1062
row['STD_CODE']: 1064
row['STD_CODE']: 1066
row['STD_CODE']: 1068
row['STD_CODE']: 1070
row['STD_CODE']: 1072
row['STD_CODE']: 1074
row['STD_CODE']: 1076
row['STD_CODE']: 1078
row['STD_CODE']: 1080
row['STD_CODE']: 1082
row['STD_CODE']: 1084
row['STD_CODE']: 1086
row['STD_CODE']: 1088
row['STD_C

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>